# Create Chat Invitations

To run, make sure you have your client PW stored as an env var locally, or replace with a hardcoded string. Also make sure to modify the `venue_ids` var with the list of venues you want to enable chat in.

In [16]:
# Imports

from openreview.api import Invitation
from openreview.api import OpenReviewClient
from openreview import tools
from os import getenv

In [17]:
# Initialize clients

local_client = OpenReviewClient(baseurl='http://localhost:3001', username='OpenReview.net', password=getenv('OR_SUPER_USER_PW'))
# dev_client = OpenReviewClient(baseurl='https://devapi2.openreview.net', username='OpenReview.net', password=getenv('OR_DEV_SUPER_USER_PW'))
# live_client = OpenReviewClient(baseurl='https://api2.openreview.net', username='OpenReview.net', password=getenv('OR_SUPER_USER_PW'))

In [19]:
# Config vars

venue_ids = ['ICML.cc/2023/Workshop/IMLH']
chat_invitation_name = 'Chat'
client = local_client

In [ ]:
# Post invitation edits

for venue_id in venue_ids:
    print(f'Starting deployment for {venue_id}...')

    # Get group info from content
    group = client.get_group(venue_id)
    if not group:
        print(f'{venue_id} not found')
        continue
    
    # Post forum tab config
    print('Posting submission invitation edit')

    submission_name = group.content.get('submission_name', {}).get('value')
    submission_group_id = f'{venue_id}/{submission_name}${{note.number}}'
    chat_invitation_id = f'{submission_group_id}/-/{chat_invitation_name}'
    review_name = group.content.get('review_name', {}).get('value')
    review_invitation_id = f'{submission_group_id}/-/{review_name}'
    client.post_invitation_edit(
        invitations=f'{venue_id}/-/Edit',
        readers = [venue_id],
        writers = [venue_id],
        signatures = [venue_id],
        invitation = Invitation(
            id = f'{venue_id}/-/{submission_name}',
            reply_forum_views = [
                {
                    'id': 'all',
                    'label': 'All'
                },
                {
                    'id': 'discussion',
                    'label': 'Discussion',
                    'filter': f'-invitations:{chat_invitation_id}',
                    'nesting': 3,
                    'sort': 'date-desc',
                    'layout': 'default',
                },
                {
                    'id': 'reviewers-chat',
                    'label': 'Reviewers Chat',
                    'filter': f'invitations:{chat_invitation_id},{review_invitation_id}',
                    'nesting': 1,
                    'sort': 'date-asc',
                    'layout': 'chat',
                    'live': True,
                    'expandedInvitations': [chat_invitation_id]
                }
            ]
        )
    )

    # Get all submissions
    submission_notes = client.get_all_notes(content={ 'venueid': f'{venue_id}/{submission_name}' })
    
    reviewers_name = group.content.get('reviewers_name', {}).get('value')
    reviewers_anon_name = group.content.get('reviewers_anon_name', {}).get('value')
    acs_name = group.content.get('area_chairs_name', {}).get('value')
    acs_anon_name = group.content.get('area_chairs_anon_name', {}).get('value')

    def post(note):
        submission_group_id = f'{venue_id}/{submission_name}{note.number}'
        submission_group = tools.get_group(client, submission_group_id)
        if submission_group is None:
            print(f'Skipping submission number {note.number} because submission group does not exist')
            return

        invitees = [
            f'{venue_id}/Program_Chairs', 
            f'{submission_group_id}/{reviewers_name}',
        ]
        if (acs_name):
            invitees.append(f'{submission_group_id}/{acs_name}')
        signatures = [
            f'{venue_id}/Program_Chairs',
            f'{submission_group_id}/{reviewers_anon_name}.*',
        ]        
        if (acs_anon_name):
            signatures.append(f'{submission_group_id}/{acs_anon_name}.*')

        return client.post_invitation_edit(
            invitations = f'{venue_id}/-/Edit',
            readers = [venue_id],
            writers = [venue_id],
            signatures = [venue_id],
            invitation = Invitation(
                id = f'{submission_group_id}/-/{chat_invitation_name}',
                readers = ['everyone'],
                writers = [venue_id],
                signatures = [venue_id],
                invitees = invitees,
                edit = {
                    'readers': [venue_id, '${2/signatures}'],
                    'writers': [venue_id],
                    'signatures': {
                        'param': {
                            'enum': signatures
                        }
                    },
                    'note': {
                        'id': {
                            'param': {
                                'withInvitation': f'{submission_group_id}/-/{chat_invitation_name}',
                                'optional': True
                            }
                        },
                        'readers': invitees,
                        'writers': [venue_id],
                        'signatures': ['${3/signatures}'],
                        'ddate': {
                            'param': {
                                'range': [ 0, 9999999999999 ],
                                'optional': True,
                                'deletable': True
                            }
                        },
                        'forum': note.id,
                        'replyto': {
                            'param': {
                                'withForum': note.id
                            }
                        },
                        'content': {
                            'message': {
                                'value': {
                                    'param': {
                                        'type': 'string',
                                        'maxLength': 5000,
                                        'markdown': True
                                    }
                                }
                            }
                        }
                    }
                }
            )
        )             
    
    tools.concurrent_requests(post, submission_notes, 'Posting chat invitations for submissions')
    print('Done')
